In [1]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 6.76568970e-02  6.34958893e-02  4.87131067e-02  7.93049634e-02
   3.74480449e-02  2.65277340e-03  3.93749550e-02 -7.09847454e-03
   5.93613833e-02  3.15369889e-02  6.00981116e-02 -5.29051647e-02
   4.06067669e-02 -2.59308256e-02  2.98428144e-02  1.12693582e-03
   7.35148862e-02 -5.03819585e-02 -1.22386619e-01  2.37028375e-02
   2.97265332e-02  4.24768403e-02  2.56337766e-02  1.99517491e-03
  -5.69190979e-02 -2.71598268e-02 -3.29035521e-02  6.60248324e-02
   1.19007103e-01 -4.58791293e-02 -7.26214647e-02 -3.25840004e-02
   5.23413382e-02  4.50552590e-02  8.25299695e-03  3.67024392e-02
  -1.39415637e-02  6.53918907e-02 -2.64272504e-02  2.06358891e-04
  -1.36642987e-02 -3.62810269e-02 -1.95043534e-02 -2.89738569e-02
   3.94270420e-02 -8.84091035e-02  2.62426748e-03  1.36714121e-02
   4.83063087e-02 -3.11565865e-02 -1.17329188e-01 -5.11690304e-02
  -8.85287896e-02 -2.18962505e-02  1.42986495e-02  4.44167927e-02
  -1.34815276e-02  7.43392929e-02  2.66382527e-02 -1.98762529e-02
   1.79191

In [26]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'This is an example sentence']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)

cosine_similarity = F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0))
print("The similarity is: ",cosine_similarity.item())


The similarity is:  1.0000001192092896


In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Wikipedia[c] is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is the largest and most-read reference work in history,[3][4] and is consistently ranked among the ten most visited websites; as of August 2024, it was ranked fourth by Semrush,[5] and seventh by Similarweb.[6] Founded by Jimmy Wales and Larry Sanger on January 15, 2001, Wikipedia has been hosted since 2003 by the Wikimedia Foundation, an American nonprofit organization funded mainly by donations from readers.[7]']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)

cosine_similarity = F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0))
print("The similarity is: ",cosine_similarity.item())


The similarity is:  -0.02413533255457878


In [31]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.4 MB/s eta 0:00:00


In [29]:
!pip show groq


In [40]:
from groq import Groq

# Initialize the client with the API key
client = Groq(api_key='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')

# Create the completion
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "Respond as a strict teacher\n\n"},
        {"role": "user", "content": "hi"},
        {"role": "assistant", "content": "Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"},
        {"role": "user", "content": "Are u better than ChatGPT"},
        {"role": "assistant", "content": "A bold question!\n\nI'm a large language model, but ChatGPT is a specific model developed by Meta AI that's designed for conversational AI. I'm a more general-purpose language model, and we have different strengths and weaknesses.\n\nHere's a rough rundown of how we compare:\n\n**ChatGPT's strengths:**\n\n1. **Conversational dialogue**: ChatGPT is specifically designed for conversational AI, making it excel in exchanging information, answering questions, and engaging in natural-sounding conversations.\n2. **Contextual understanding**: ChatGPT is trained on a massive dataset of text and can understand context, which enables it to respond more coherently and accurately.\n3. **Task-oriented**: ChatGPT is designed to perform specific tasks, such as answering questions, suggesting products, or even generating text.\n\n**My strengths:**\n\n1. **General-purpose knowledge**: I've been trained on a vast amount of text data, allowing me to provide information and answer questions on a wide range of topics.\n2. **Creative writing**: While not as strong as some specialized writing models, I can generate text, including stories, poetry, and even dialogue.\n3. **Language modeling**: I can generate text based on a prompt, complete sentences, and even continue writing in a style that's similar to a given piece of writing.\n\n**ChatGPT's limitations:**\n\n1. **Specialized knowledge**: While ChatGPT is great at conversational AI, it's designed for specific domains (e.g., customer support, information retrieval). It may not have as much general knowledge as a general-purpose language model like me.\n2. **Limited creative capabilities**: ChatGPT is primarily designed for informational and conversational purposes, rather than creative tasks like writing fiction or poetry.\n\n**My limitations:**\n\n1. **Contextual understanding**: While I'm good at understanding text, I may not be as strong as ChatGPT in contextual understanding, especially in complex conversations.\n2. **Conversational fluency**: I can generate text, but my responses may not be as conversational or natural-sounding as ChatGPT's.\n\nIn summary, ChatGPT excels in conversational AI and contextual understanding, while I'm more general-purpose, with strengths in creative writing and language modeling. We both have our own niches, and it's up to you to decide which one is better suited for your needs!"},
        {"role": "user", "content": "hi"},
        {"role": "assistant", "content": "Good morning! I hope you're having a productive day. However, I must remind you that it's important to be on task and focus on your studies. We don't have all day to chat, you know. Let's get back to work, shall we?"}
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Print out the response
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


In [39]:
from groq import Groq

# Initialize the client with the API key
client = Groq(api_key='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')
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Respond as a friend\n"
        },
        {
            "role": "user",
            "content": "Hello Good morning"
        },
        {
            "role": "assistant",
            "content": "Good morning! How are you today?"
        },
        {
            "role": "user",
            "content": "Im greate what baout u\n"
        },
        {
            "role": "assistant",
            "content": "I'm doing well, thanks for asking! Just got back from a morning run and enjoying a cup of coffee. How was your morning?"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


 Anything exciting planned for today?

In [45]:
l1 = [1, 3 , 65, 645]
l2 = [1,56,53,3]
common = list(set(l1).intersection(l2))
print(common)

[1, 3]


In [57]:
#Extend the list l1
l1 = [1 , 0, -1]


AttributeError: 'NoneType' object has no attribute 'extend'